In [ ]:
import pandas as pd

In [ ]:
class Player:
    def __init__(self, ID, name, photo, nationality, flag, club, club_logo):
        self.ID = ID
        self.name = name
        self.photo = photo
        self.nationality = nationality
        self.flag = flag
        self.club = club
        self.club_logo = club_logo
    
    def __lt__(self, other):
        return self.ID < other.ID

In [ ]:
def read_csv(filename = './assets/data.csv'): 
    df = pd.read_csv(filename, usecols=['ID', 'Name', 'Photo', 'Nationality', 'Flag', 'Club', 'Club Logo'])
    
    players = []
    for index, row in df.iterrows():
        ID = row['ID']
        name = row['Name']
        photo = row['Photo']
        nationality = row['Nationality']
        flag = row['Flag']
        club = row['Club']
        club_logo = row['Club Logo']
        
        player = Player(ID, name, photo, nationality, flag, club, club_logo)
        players.append(player)
        
    players.sort()
    
    return players

In [ ]:
players = read_csv()